# Titanic
## EDA
- RandomForest 와 ANN 돌리기 전 데이터를 분석 및 Vector 생성

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from preprocess import *


In [2]:
# train_data = pd.read_csv('/Users/hj/Desktop/ESTCODE/train.csv')
# test_data = pd.read_csv('/Users/hj/Desktop/ESTCODE/test.csv')

train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
train_data=process(train_data)
test_data =process2(test_data)
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,is_baby,name_title,family
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Mr,Single
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,Mrs,Single
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0,Miss,Single
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0,Mrs,Single
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0,Mr,Single


In [3]:
#데이터 처리하기, 모양 찍어보기
X_train = get_X(train_data,["Pclass", "Sex", "family", "is_baby", "name_title"])
#모델 저장하기 
y_train = get_y(train_data)
X_test = get_X(test_data, ["Pclass", "Sex", "family", "is_baby", "name_title"])

X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)


X_train.shape, y_train.shape

((891, 14), (891, 1))

# Random Forest

In [4]:
#Random Forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
import warnings
warnings.simplefilter("ignore")


scoring = ['accuracy', 'precision', 'recall', 'f1']
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=2023)
scores = cross_validate(clf, X_train, y_train, scoring=scoring, cv=5)
scores_df = pd.DataFrame(scores)
pd.concat([scores_df, scores_df.apply(['mean', 'std'])])


,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1
0,0.125854,0.027152,0.804469,0.765625,0.710145,0.736842
1,0.121064,0.032621,0.831461,0.771429,0.794118,0.782609
2,0.125207,0.026264,0.814607,0.830189,0.647059,0.727273
3,0.133124,0.025092,0.803371,0.789474,0.661765,0.720000
4,0.115393,0.024966,0.842697,0.847458,0.724638,0.781250
mean,0.124129,0.027219,0.819321,0.800835,0.707545,0.749595
std,0.006534,0.003150,0.017255,0.036293,0.058198,0.030119


In [5]:
from sklearn.model_selection import StratifiedKFold, KFold
#from tqdm.auto import tqdm
from torch.utils.data import TensorDataset
from sklearn.metrics import accuracy_score, f1_score

clf2 = RandomForestClassifier(n_estimators=100, max_depth=3, random_state=2023)


kf = KFold(n_splits = 5, shuffle = True, random_state = 1111)
model_acc = []
model_f1 = []
# print(y_train.shape)

ind = 0
for train_index, test_index in kf.split(X_train,y_train):
		#나누기
    X_train_k, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_k, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
		
		#모델 훈련하기
    clf2.fit(X_train_k, y_train_k)

		#예측값 담기
    y_pred = clf2.predict(X_val)

    print(f'accuarcy: {accuracy_score(y_pred, y_val)},f1_score: {f1_score(y_pred, y_val)}')
    model_acc.append(accuracy_score(y_pred, y_val))
    model_f1.append(f1_score(y_pred, y_val))

print(np.mean(model_acc))
print(np.mean(model_f1))

accuarcy: 0.8268156424581006,f1_score: 0.7891156462585033
accuarcy: 0.7696629213483146,f1_score: 0.672
accuarcy: 0.8820224719101124,f1_score: 0.8141592920353982
accuarcy: 0.7808988764044944,f1_score: 0.7022900763358779
accuarcy: 0.7808988764044944,f1_score: 0.7194244604316548
0.8080597577051034
0.7393978950122868


In [6]:
y_pred = clf2.predict(X_test)
y_pred =y_pred.astype('int')
y_pred

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,

In [7]:

#submission
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")


Your submission was successfully saved!


# RandomForestRegressor

In [8]:
'''
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


scoring = ['accuracy','precision','recall','f1']
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

model2_acc = []
model2_f1 = []
# print(y_train.shape)

ind = 0
for train_index, test_index in kf.split(X_train, y_train):
		#나누기
    X_train_k, X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_k, y_val = y_train.iloc[train_index], y_train.iloc[test_index]
		
		#모델 훈련하기
    rf_regressor.fit(X_train_k, y_train_k)

		#예측값 담기
    y_pred = rf_regressor.predict(X_val)

    print(f'f1_score: {f1_score(y_pred, y_val)}')
    #model2_acc.append(accuracy_score(y_pred, y_val))
    model2_f1.append(f1_score(y_pred, y_val))
#print(np.mean(model2_acc))
print(np.mean(model2_f1))
'''


"\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.metrics import mean_squared_error\n\n\nscoring = ['accuracy','precision','recall','f1']\nrf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)\n\nmodel2_acc = []\nmodel2_f1 = []\n# print(y_train.shape)\n\nind = 0\nfor train_index, test_index in kf.split(X_train, y_train):\n\t\t#나누기\n    X_train_k, X_val = X_train.iloc[train_index], X_train.iloc[test_index]\n    y_train_k, y_val = y_train.iloc[train_index], y_train.iloc[test_index]\n\t\t\n\t\t#모델 훈련하기\n    rf_regressor.fit(X_train_k, y_train_k)\n\n\t\t#예측값 담기\n    y_pred = rf_regressor.predict(X_val)\n\n    print(f'f1_score: {f1_score(y_pred, y_val)}')\n    #model2_acc.append(accuracy_score(y_pred, y_val))\n    model2_f1.append(f1_score(y_pred, y_val))\n#print(np.mean(model2_acc))\nprint(np.mean(model2_f1))\n"

In [9]:
'''
output2 = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': y_pred})
output2.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")
'''

'\noutput2 = pd.DataFrame({\'PassengerId\': test_data.PassengerId, \'Survived\': y_pred})\noutput2.to_csv(\'submission.csv\', index=False)\nprint("Your submission was successfully saved!")\n'